In [8]:
import pandas as pd
import numpy as np
import os
from dshortcuts.utils import open_pickle, save_pickle
# from operator import itemgetter
import geocoder

In [9]:
os.listdir('../input')

['test.csv',
 'sample_submission.csv',
 'item_categories.csv',
 'trainDtypes.pickle',
 'sales_train.csv',
 'itemCategoriesDtypes.pickle',
 'items.csv',
 'shops.csv']

In [10]:
# open prepared dtypes to save memory
trainDtypes = open_pickle(os.path.join('../input', 'trainDtypes.pickle'))
itemCategoriesDtypes = open_pickle(os.path.join('../input', 'itemCategoriesDtypes.pickle'))

In [11]:
train = pd.read_csv('../input/sales_train.csv', dtype=trainDtypes, parse_dates=['date'])
itemCategories = pd.read_csv('../input/item_categories.csv', dtype=itemCategoriesDtypes)
test = pd.read_csv('../input/test.csv')
shops = pd.read_csv('../input/shops.csv')
items = pd.read_csv('../input/items.csv')

In [12]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.000000,1.0
1,2013-03-01,0,25,2552,899.000000,1.0
2,2013-05-01,0,25,2552,899.000000,-1.0
3,2013-06-01,0,25,2554,1709.050049,1.0
4,2013-01-15,0,25,2555,1099.000000,1.0


##### Categories
1. Are they broad enough?
2. Are they descriptive enough?

In [13]:
itemCategories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


too descriptive. At first look
we may try to decouple broad categories by `split('-')`

In [14]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [15]:
train = train.merge(items.drop('item_name', axis=1), on='item_id', how='left')

#### Obvious features: 

- ~~day~~
- month
- quarter
- year
- ~~dayofweek~~
- ~~is workingday~~
- ~~delta from every 22th day of month (~salary day in Rus)~~
- ~~delta from every 7th day of month (~pre-salary day in Rus)~~

I considered that in test set there is only a month that we know from a datetime features. It means that we can get only month+ features

In [16]:
# train['day'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['quarter'] = train['date'].dt.quarter
# train['dayofweek'] = train['date'].dt.weekday
# train['is_weekend'] = (train['dayofweek'] >= 5).astype('int8')

In [17]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,month,year,quarter
0,2013-02-01,0,59,22154,999.000000,1.0,37,2,2013,1
1,2013-03-01,0,25,2552,899.000000,1.0,58,3,2013,1
2,2013-05-01,0,25,2552,899.000000,-1.0,58,5,2013,2
3,2013-06-01,0,25,2554,1709.050049,1.0,58,6,2013,2
4,2013-01-15,0,25,2555,1099.000000,1.0,56,1,2013,1


#### Sales & Price features: (think more...)

**H1**:
! People buy things when the price goes down

- 1quart price in previous month (shop-item)
- 3quart price in previous month (shop-item)
- price delta in prev month (shop-item)
- price delta over 2 months (shop-item)
- monthly sales by {shop_id, category_id}
- monthly sales by {shop_id, item_id}
- monthly sales by {category_id}

Probably it worth to make a **H2**:
! People buy when a shop signals about sales (no matter which item)

In [19]:
shop_item = train.groupby(['date_block_num', 'shop_id', 'item_id'])

In [20]:
q25 = shop_item.item_price.quantile(.25)
q75 = shop_item.item_price.quantile(.75)

In [21]:
q25.reset_index()

,date_block_num,shop_id,item_id,item_price
0,0,0,32,221.00
1,0,0,33,347.00
2,0,0,35,247.00
3,0,0,43,221.00
4,0,0,51,127.75
...,...,...,...,...
1609119,33,59,22087,119.00
1609120,33,59,22088,119.00
1609121,33,59,22091,179.00
1609122,33,59,22100,629.00


In [33]:
q25.reset_index().date_block_num + 1

0           1
1           1
2           1
3           1
4           1
           ..
1609119    34
1609120    34
1609121    34
1609122    34
1609123    34
Name: date_block_num, Length: 1609124, dtype: int64

In [29]:
train.merge(
    q25.reset_index().rename({'item_price': 'mon_price_1qrt'}, axis=1), 
    on=['date_block_num', 'shop_id', 'item_id'])

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,month,year,quarter,mon_price_1qrt
0,2013-02-01,0,59,22154,999.000000,1.0,37,2,2013,1,999.000000
1,2013-03-01,0,25,2552,899.000000,1.0,58,3,2013,1,899.000000
2,2013-05-01,0,25,2552,899.000000,-1.0,58,5,2013,2,899.000000
3,2013-06-01,0,25,2554,1709.050049,1.0,58,6,2013,2,1709.050049
4,2013-01-15,0,25,2555,1099.000000,1.0,56,1,2013,1,1099.000000
...,...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.000000,1.0,55,10,2015,4,299.000000
2935845,2015-11-10,33,25,7393,349.000000,1.0,55,11,2015,4,349.000000
2935846,2015-10-10,33,25,7384,749.000000,1.0,55,10,2015,4,749.000000
2935847,2015-10-14,33,25,7459,349.000000,1.0,55,10,2015,4,349.000000


In [28]:
q25.reset_index().rename({'item_price': 'mon_price_1qrt'}, axis=1)

,date_block_num,shop_id,item_id,mon_price_1qrt
0,0,0,32,221.00
1,0,0,33,347.00
2,0,0,35,247.00
3,0,0,43,221.00
4,0,0,51,127.75
...,...,...,...,...
1609119,33,59,22087,119.00
1609120,33,59,22088,119.00
1609121,33,59,22091,179.00
1609122,33,59,22100,629.00


In [27]:
pd.DataFrame.rename?

In [23]:
shop_item.item_price.quantile(.75)

shop_id  item_id
0        30         265.0
         31         434.0
         32         221.0
         33         347.0
         35         247.0
                    ...  
59       22154      999.0
         22155      149.0
         22162      399.0
         22164      749.0
         22167      299.0
Name: item_price, Length: 424124, dtype: float64

In [26]:
((shop_item.item_price.quantile(.25) - shop_item.item_price.quantile(.75)) != 0)

shop_id  item_id
0        30         False
         31         False
         32         False
         33         False
         35         False
                    ...  
59       22154      False
         22155      False
         22162      False
         22164       True
         22167      False
Name: item_price, Length: 424124, dtype: bool

#### Geographical features
1. Get the city from the shop info
2. Try to get the lat lon of the shop with map api ?

-------------

- what's next?

In [90]:
def extract_city(value):
    return value.split()[0].strip('!')

def yandex_latlong(city):
    return geocoder.google(city).latlng


shops['city'] = shops['shop_name'].apply(extract_city)

In [94]:
# because of the error with geocoder.yandex, i took ready latlong from here:
# https://www.kaggle.com/bourov/yada-yet-another-e-da-by-geo-location-of-shops/data?select=coordinates.csv
coordinates = pd.read_csv('../input/coordinates.csv', index_col=0)
coordinates.head()

,lat,long
shop_id,,
0,62.028103,129.732663
1,62.028103,129.732663
2,44.429866,40.252969
3,55.796339,37.938199
4,48.786293,44.751867


In [99]:
shops = shops.merge(coordinates, left_on='shop_id', right_index=True)
shops

,shop_name,shop_id,city,latlong,lat_x,long_x,lat_y,long_y,lat,long
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,None,62.028103,129.732663,62.028103,129.732663,62.028103,129.732663
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,None,62.028103,129.732663,62.028103,129.732663,62.028103,129.732663
2,"Адыгея ТЦ ""Мега""",2,Адыгея,None,44.429866,40.252969,44.429866,40.252969,44.429866,40.252969
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,None,55.796339,37.938199,55.796339,37.938199,55.796339,37.938199
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,None,48.786293,44.751867,48.786293,44.751867,48.786293,44.751867
5,"Вологда ТРЦ ""Мармелад""",5,Вологда,None,59.220473,39.891559,59.220473,39.891559,59.220473,39.891559
6,"Воронеж (Плехановская, 13)",6,Воронеж,None,51.661535,39.200287,51.661535,39.200287,51.661535,39.200287
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж,None,51.661535,39.200287,51.661535,39.200287,51.661535,39.200287
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж,None,51.661535,39.200287,51.661535,39.200287,51.661535,39.200287
9,Выездная Торговля,9,Выездная,None,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
shops.groupby('city').size().describe()

count    31.000000
mean      1.935484
std       2.205077
min       1.000000
25%       1.000000
50%       1.000000
75%       2.000000
max      13.000000
dtype: float64

##### C&C: Cleaning &Clustering
Let's try to cleanup the NaNs, indicate them with a new column, and add clusters with kNN (aka Regions)

In [ ]:
train.groupby(['date_block_num', 'shop_id', 'item_id']).item_cnt_day.sum()

#### Validation

- is it just a timewise split?
- do we have new shops / items in test set? (if yes - we have to mimic this in validation strategy also)

In [53]:
# shops in test but not in train (unseen)
_mask = ~ np.isin(test.shop_id.unique(), train.shop_id.unique())
test.shop_id.unique()[_mask]

array([], dtype=int64)

In [61]:
# items in test but not in train (unseen)
_mask = ~ np.isin(test.item_id.unique(), train.item_id.unique())
test.item_id.unique()[_mask].shape,  test.item_id.unique().shape

((363,), (5100,))

Let's do the same with a validation set

In [63]:
_mask = ~ np.isin(valid.shop_id.unique(), train[train.date_block_num < 33].shop_id.unique())
valid.shop_id.unique()[_mask]

array([36], dtype=int8)

In [66]:
_mask = ~ np.isin(valid.item_id.unique(), train[train.date_block_num < 33].item_id.unique())
valid.item_id.unique()[_mask].shape, valid.item_id.unique().shape

((475,), (5413,))

Well, the test set is roughly the same as validation. We can use validation set as `date_block_num == 33` for sure

Probably it's a good idea to exclude `shop_id == 36` to mimic a test set (the last has no new shops)

One more **TODO**:
check if we can use 2 folds: 33's and 32's blocks

In [6]:
valid = train[train['date_block_num'] == (train['date_block_num'].max())]

In [ ]:
(test['shop_id'].isin(train['shop_id']) == False).sum(), \
(test['item_id'].isin(train['item_id']) == False).sum()

In [ ]:
test.head()